In [1]:
!pip install python-telegram-bot
!pip install -U ipykernel

     |████████████████████████████████| 358kB 10.7MB/s 
     |████████████████████████████████| 2.3MB 49.2MB/s 
     |████████████████████████████████| 491kB 78.2MB/s 
  Created wheel for tornado: filename=tornado-6.0.3-cp36-cp36m-linux_x86_64.whl size=423198 sha256=bc854da20e03af84296c375c9baf9489371d5e5d2e830cda99a255ae54312de8
  Stored in directory: /root/.cache/pip/wheels/84/bf/40/2f6ef700f48401ca40e5e3dd7d0e3c0a90e064897b7fe5fc08
Successfully built tornado
ERROR: google-colab 1.0.0 has requirement tornado~=4.5.0, but you'll have tornado 6.0.3 which is incompatible.
  Found existing installation: tornado 4.5.3
    Uninstalling tornado-4.5.3:
      Successfully uninstalled tornado-4.5.3


     |████████████████████████████████| 122kB 8.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.6.0, but you'll have ipykernel 5.1.4 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=4.5.0, but you'll have tornado 6.0.3 which is incompatible.
  Found existing installation: ipykernel 4.6.1
    Uninstalling ipykernel-4.6.1:
      Successfully uninstalled ipykernel-4.6.1


# model

In [0]:
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from scipy import misc
import copy
import numpy as np 

import os
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler, ConversationHandler, CallbackQueryHandler
from telegram.ext.dispatcher import run_async
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
import logging


In [0]:
class ContentLoss(nn.Module):

    def __init__(self, target, ):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()  # это константа. Убираем ее из дерева вычеслений
        self.loss = F.mse_loss(self.target, self.target)  # to initialize with something

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [0]:
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = self.gram_matrix(target_feature).detach()
        self.loss = F.mse_loss(self.target, self.target)  # to initialize with something

    def forward(self, input):
        G = self.gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

    def gram_matrix(self,input):
        batch_size, h, w, f_map_num = input.size()  # batch size(=1)
        # b=number of feature maps
        # (h,w)=dimensions of a feature map (N=h*w)

        features = input.view(batch_size * h, w * f_map_num)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(batch_size * h * w * f_map_num)

In [0]:
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

In [0]:
class StyleTransferModel:
    def __init__(self):
        # Сюда необходимо перенести всю иницализацию, вроде загрузки свеерточной сети и т.д.
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.cnn = models.vgg19(pretrained=True).features.to(device).eval()
        pass


    def transfer_style(self, content_img_stream, style_img_stream, num_steps=500,
                        style_weight=100000000, content_weight=1):
        content_img = self.process_image(content_img_stream)
        style_img = self.process_image(style_img_stream)
        input_img = content_img.clone().detach()
        """Run the style transfer."""
        print('Building the style transfer model..')
        model, style_losses, content_losses = self.get_style_model_and_losses(style_img,
                                                                         content_img,)
        optimizer = self.get_input_optimizer(input_img)

        print('Optimizing..')
        run = [0]
        while run[0] <= num_steps:

            def closure():
                # correct the values
                # это для того, чтобы значения тензора картинки не выходили за пределы [0;1]
                input_img.data.clamp_(0, 1)

                optimizer.zero_grad()

                model(input_img)

                style_score = 0
                content_score = 0

                for sl in style_losses:
                    style_score += sl.loss
                for cl in content_losses:
                    content_score += cl.loss

                # взвешивание ощибки
                style_score *= style_weight
                content_score *= content_weight

                loss = style_score + content_score
                loss.backward()

                run[0] += 1
                if run[0] % 50 == 0:
                    print("run {}:".format(run))
                    print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                        style_score.item(), content_score.item()))
                    print()

                return style_score + content_score

            optimizer.step(closure)

        # a last correction...
        input_img.data.clamp_(0, 1)
        image = input_img.cpu().clone().detach()
        unloader = transforms.ToPILImage()
        return unloader(image[0])


    def get_input_optimizer(self, input_img):
        #добоваляет содержимое тензора катринки в список изменяемых оптимизатором параметров
        optimizer = optim.LBFGS([input_img.requires_grad_()]) 
        return optimizer


    def process_image(self, img_stream):
        # TODO размер картинки, device и трансформации не меняются в течении всей работы модели,
        # поэтому их нужно перенести в конструктор!
        imsize = 720
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(device)
        loader = transforms.Compose([
            transforms.Resize(imsize),  # нормируем размер изображения
            transforms.CenterCrop(imsize),
            transforms.ToTensor()])  # превращаем в удобный формат

        image = Image.open(img_stream)
        image = loader(image).unsqueeze(0)
        return image.to(device, torch.float)

    def get_style_model_and_losses(self, style_img, content_img):
        content_layers = ['conv_4']
        style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
        cnn = copy.deepcopy(self.cnn)

        # normalization module

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
        normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)
        normalization = Normalization(normalization_mean, normalization_std).to(device)

        # just in order to have an iterable access to or list of content/syle
        # losses
        content_losses = []
        style_losses = []

        # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
        # to put in modules that are supposed to be activated sequentially
        model = nn.Sequential(normalization)

        i = 0  # increment every time we see a conv
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = 'conv_{}'.format(i)
            elif isinstance(layer, nn.ReLU):
                name = 'relu_{}'.format(i)
                # Переопределим relu уровень
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = 'pool_{}'.format(i)
            elif isinstance(layer, nn.BatchNorm2d):
                name = 'bn_{}'.format(i)
            else:
                raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).detach()
                content_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).detach()
                style_loss = StyleLoss(target_feature)
                model.add_module("style_loss_{}".format(i), style_loss)
                style_losses.append(style_loss)

        # выбрасываем все уровни после последенего styel loss или content loss
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, style_losses, content_losses

# CycleGan

In [6]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2194, done.
remote: Total 2194 (delta 0), reused 0 (delta 0), pack-reused 2194
Receiving objects: 100% (2194/2194), 8.01 MiB | 10.60 MiB/s, done.
Resolving deltas: 100% (1423/1423), done.


In [0]:
os.chdir('/content/pytorch-CycleGAN-and-pix2pix/')

In [0]:
!pip install -r requirements.txt

In [10]:
!bash ./scripts/download_cyclegan_model.sh winter2summer_yosemite

Note: available models are apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower
Specified [winter2summer_yosemite]
for details.

--2020-02-08 12:31:36--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/winter2summer_yosemite.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.189.73
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45575747 (43M)
Saving to: ‘./checkpoints/winter2summer_yosemite_pretrained/latest_net_G.pth’

./checkpoints/winte 100%[===================>]  43.46M  13.9MB/s    in 3.1s    

2020-02-08 12:31:39 (13.9 MB/s) - ‘./checkpoints/winter2summer_yosemite_pretrained/latest_ne

# main

In [0]:
from io import BytesIO
import subprocess
import shutil

In [12]:
# В бейзлайне пример того, как мы можем обрабатывать две картинки, пришедшие от пользователя.
model = StyleTransferModel()
first_image_file = {}

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth


In [0]:
def send_prediction_on_photo(update, context):
    chat_id = update.message.chat_id
    print("Got image from {}".format(chat_id))
    

    image_file = update.message.photo[-1].get_file()

    if chat_id in first_image_file:
        update.message.reply_text('Принял вторую фотографию.Ожидайте 2 минуты')
        # первая картинка, которая к нам пришла станет content image, а вторая style image
        content_image_stream = BytesIO()
        first_image_file[chat_id].download(out=content_image_stream)
        del first_image_file[chat_id]

        style_image_stream = BytesIO()
        image_file.download(out=style_image_stream)
        del image_file

        output = model.transfer_style(content_image_stream, style_image_stream)

        # теперь отправим назад фото
        output_stream = BytesIO()
        output.save(output_stream, format='PNG')
        output_stream.seek(0)
        update.message.reply_photo(photo=output_stream)
        print("Sent Photo to user")
        update.message.reply_text('Пока! Если хочешь еще,напиши "/start".')
        return ConversationHandler.END
    else:
        update.message.reply_text('Принял фото')
        first_image_file[chat_id] = image_file
        return PHOTO_1
    
    

In [0]:
def send_prediction_on_photo_CycleGAN(update, context):
    os.chdir('/content/pytorch-CycleGAN-and-pix2pix/')
    chat_id = update.message.chat_id
    print("Got image from {}".format(chat_id))
    update.message.reply_text('Принял фото')

    os.mkdir(str(chat_id))

    image_file = update.message.photo[-1].get_file()
    first_image_file[chat_id] = image_file
    
    image_path = str(chat_id)
    first_image_file[chat_id].download(image_path+'/image.png')
    del first_image_file[chat_id]

    subprocess.call(['python','/content/pytorch-CycleGAN-and-pix2pix/test.py',
    '--dataroot',image_path,'--name','winter2summer_yosemite_pretrained','--model','test',
    '--no_dropout','--results','/content/'+image_path,'--load_size','720','--crop_size','720'])

    shutil.rmtree('/content/pytorch-CycleGAN-and-pix2pix/' + image_path)
    
    output = Image.open('/content/'+image_path+ '/winter2summer_yosemite_pretrained/test_latest/images/image_fake.png')

    output_stream = BytesIO()
    output.save(output_stream, format='PNG')
    output_stream.seek(0)
    update.message.reply_photo(photo=output_stream)

    shutil.rmtree('/content/' +image_path)
    print("Sent Photo to user")
    update.message.reply_text('Пока! Если хочешь еще,напиши "/start".')
    return ConversationHandler.END

In [0]:
def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Напиши мне "/style" ,и отправь две картинки .А я перенесу стиль второй картинки на первую')
    update.message.reply_text('Напиши мне "/gan" ,и отправь одну зимнюю картинку.И я изменю ее на летнюю)')
    update.message.reply_text('Остановить бота - "/stop"')
    return NEURON_CHOISE

In [0]:
def cancel(update, context):
    update.message.reply_text('Пока! Если хочешь еще,напиши "/start".')

    return ConversationHandler.END

In [0]:
NEURON_CHOISE,PHOTO_1,PHOTO_2 = range(3)

In [36]:
if __name__ == '__main__':

    token = ""
    # Включим самый базовый логгинг, чтобы видеть сообщения об ошибках
    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        level=logging.INFO)
    
    #, request_kwargs={'proxy_url': 'socks5h://163.172.152.192:1080'}
    updater = Updater(token,use_context=True)
    dp = updater.dispatcher

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            NEURON_CHOISE: [CommandHandler('Style', lambda x,y: PHOTO_1),CommandHandler('Gan', lambda x,y: PHOTO_2)],
            PHOTO_1: [MessageHandler(Filters.photo, send_prediction_on_photo),],
            PHOTO_2: [MessageHandler(Filters.photo, send_prediction_on_photo_CycleGAN),],
            
        },

        fallbacks=[CommandHandler('stop', cancel)]
    )

    start_button = InlineKeyboardButton('start')
    InlineKeyboardMarkup(start_button)
    
    dp.add_handler(conv_handler)
    run_async(conv_handler)
    updater.start_polling()

    updater.idle()

Got image from 258862724
Sent Photo to user
Got image from 258862724
Got image from 258862724
cuda
cuda
Building the style transfer model..
Optimizing..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


run [50]:
Style Loss : 75756.375000 Content Loss: 11.073621

run [100]:
Style Loss : 56887.097656 Content Loss: 16.510307

run [150]:
Style Loss : 45065.167969 Content Loss: 22.019827

run [200]:
Style Loss : 30939.123047 Content Loss: 23.791767

run [250]:
Style Loss : 1858546816.000000 Content Loss: 380.438324

run [300]:
Style Loss : 39183.773438 Content Loss: 25.071609

run [350]:
Style Loss : 17056.933594 Content Loss: 24.910055

run [400]:
Style Loss : 5569.303223 Content Loss: 24.194885

run [450]:
Style Loss : 516283616.000000 Content Loss: 221.784317

run [500]:
Style Loss : 53711.492188 Content Loss: 23.487028

Sent Photo to user
Got image from 258862724
Sent Photo to user


2020-02-08 12:48:01,055 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
